In [263]:
# Import relevant libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [264]:
# Import NICS gun data & US census data.
# We will name the NICS gun dataset 'df1'.
# We will name the Census dataset 'df2'.
df1 = pd.read_csv('gun_data.csv')
df2 = pd.read_csv('us_census_data.csv')

In [265]:
df1['month'] = pd.to_datetime(df1['month'])

In [266]:
df1.drop(['permit_recheck',
          'admin',
          'prepawn_handgun',
          'prepawn_long_gun',
          'prepawn_other',
          'redemption_handgun',
          'redemption_long_gun',
          'redemption_other',
          'returned_handgun',
          'returned_long_gun',
          'returned_other',
          'rentals_handgun',
          'rentals_long_gun',
          'private_sale_handgun',
          'private_sale_long_gun',
          'private_sale_other',
          'return_to_seller_handgun',
          'return_to_seller_long_gun',
          'return_to_seller_other'],axis=1,inplace=True)

In [268]:
df1.head()

,month,state,permit,handgun,long_gun,other,multiple,totals
0,2017-09-01,Alabama,16717.0,5734.0,6320.0,221.0,317,32019
1,2017-09-01,Alaska,209.0,2320.0,2930.0,219.0,160,6303
2,2017-09-01,Arizona,5069.0,11063.0,7946.0,920.0,631,28394
3,2017-09-01,Arkansas,2935.0,4347.0,6063.0,165.0,366,17747
4,2017-09-01,California,57839.0,37165.0,24581.0,2984.0,0,123506


In [270]:
# Check number of unique states to compare with Census dataframe
df1['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Mariana Islands', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'], dtype=object)

The `state` column in FBI dataset has 5 more states than in `state` column in the Census dataset.  We will remove data rows from the following states that only exist in the FBI dataset:
- District of Columbia,
- Guam,
- Mariana Islands,
- Puerto Rico,
- Virgin Islands

In [132]:
# Select rows which contains data from the extraneous states.
extra_states = ['District of Columbia','Guam','Mariana Islands','Puerto Rico','Virgin Islands']
extra_state_dfs = df1.loc[df1['state'].isin(extra_states)]
extra_state_dfs.head()

,month,state,permit,handgun,long_gun,other,multiple,totals
8,2017-09-01,District of Columbia,1.0,56.0,4.0,0.0,0,61
11,2017-09-01,Guam,0.0,74.0,47.0,10.0,3,135
21,2017-09-01,Mariana Islands,0.0,12.0,3.0,0.0,1,16
41,2017-09-01,Puerto Rico,0.0,619.0,123.0,17.0,22,791
49,2017-09-01,Virgin Islands,1.0,6.0,2.0,0.0,0,9


In [133]:
# Drop rows where the state is one of the extra_states series.
df1.drop(df1.index[df1['state'].isin(extra_states)], inplace=True)
df1['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [136]:
df1['state'].nunique()

50

In [271]:
df1.isnull().sum()

month          0
state          0
permit        24
handgun       20
long_gun      19
other       6985
multiple       0
totals         0
dtype: int64

In [273]:
# Convert each month data to year, as we are mainly interested in annual data.
df1['month'] = pd.DatetimeIndex(df1['month']).to_period("Y")

In [276]:
# Rename 'month' column to years, as we are now only seeing year data
df1.rename(index=str, columns={"month": "years"})

,years,state,permit,handgun,long_gun,other,multiple,totals
0,2017,Alabama,16717.0,5734.0,6320.0,221.0,317,32019
1,2017,Alaska,209.0,2320.0,2930.0,219.0,160,6303
2,2017,Arizona,5069.0,11063.0,7946.0,920.0,631,28394
3,2017,Arkansas,2935.0,4347.0,6063.0,165.0,366,17747
4,2017,California,57839.0,37165.0,24581.0,2984.0,0,123506
5,2017,Colorado,4356.0,15751.0,13448.0,1007.0,1062,35873
6,2017,Connecticut,4343.0,4834.0,1993.0,274.0,0,12117
7,2017,Delaware,275.0,1414.0,1538.0,66.0,68,3502
8,2017,District of Columbia,1.0,56.0,4.0,0.0,0,61
9,2017,Florida,10784.0,39199.0,17949.0,2319.0,1721,77390


In [280]:
# Sum each month (that has already been converted to years) to get annual sums for each state.
df1_by_year = df1.groupby(['month','state']).sum()
df1_by_year

permit   handgun  long_gun    other  multiple  \
month state                                                                   
1998  Alabama                    0.0   10567.0   25723.0      0.0       278   
      Alaska                     4.0    1365.0    2555.0      0.0        61   
      Arizona                 1733.0    7157.0    8275.0      0.0       278   
      Arkansas                   0.0    4649.0   16914.0      0.0       189   
      California              5366.0   28641.0   33438.0      0.0         0   
      Colorado                   0.0    7925.0   15233.0      0.0       560   
      Connecticut             2817.0    1521.0    2486.0      0.0         6   
      Delaware                   0.0     507.0    1618.0      0.0        10   
      District of Columbia       0.0       0.0       0.0      0.0         0   
      Florida                    0.0   11507.0   19283.0      0.0       347   
      Georgia                  473.0    1288.0    1665.0      0.0        31   
      Guam                       0.0       2.0       7.0      0.0         0   
      Hawaii                   401.0       1.0      29.0      0.0         0   
      Idaho                      0.0    1873.0    5033.0      0.0        91   
      Illinois               24026.0    6207.0   11050.0      0.0       194   
      Indiana                    0.0    5357.0   17143.0      0.0        18   
      Iowa                    7835.0      66.0    6738.0      0.0         7   
      Kansas                     0.0    3225.0    9011.0      0.0       150   
      Kentucky                   0.0   11243.0   20206.0      0.0       420   
      Louisiana                  0.0    7572.0   19860.0      0.0       210   
      Maine                      0.0    1210.0    2848.0      0.0        48   
      Mariana Islands            0.0       0.0       0.0      0.0         0   
      Maryland                1638.0     953.0    6405.0      0.0        11   
      Massachusetts              0.0     321.0    1116.0      0.0         6   
      Michigan               10086.0     127.0   20373.0      0.0        10   
      Minnesota               1999.0    1105.0    8967.0      0.0        45   
      Mississippi                0.0    7124.0   18771.0      0.0       202   
      Missouri                   0.0    4212.0   19753.0      0.0       122   
      Montana                    0.0    1795.0    4525.0      0.0        86   
      Nebraska                1925.0      91.0    4633.0      0.0         6   
...                              ...       ...       ...      ...       ...   
2017  Minnesota             346901.0   78202.0   88846.0   5998.0      3198   
      Mississippi            16293.0   73897.0   48224.0   2415.0      3585   
      Missouri                7863.0  183064.0  123928.0  10977.0      9285   
      Montana                13773.0   24541.0   34576.0   1547.0      1952   
      Nebraska               33172.0    1138.0   14619.0    166.0        38   
      Nevada                 20903.0   41577.0   22406.0   2304.0      2276   
      New Hampshire          20097.0   46612.0   25961.0   1393.0        37   
      New Jersey                 0.0   47381.0   28015.0   1717.0         0   
      New Mexico             10389.0   49729.0   35230.0   3745.0      2750   
      New York               26316.0   96405.0  122593.0   9192.0      1650   
      North Carolina        228436.0   12055.0   99717.0   6223.0      2092   
      North Dakota            9729.0   14383.0   20292.0    895.0       639   
      Ohio                  109493.0  244699.0  139129.0  14060.0     10319   
      Oklahoma                   0.0  102785.0   77253.0   8495.0      7684   
      Oregon                 25367.0  122185.0   92090.0      0.0        17   
      Pennsylvania          233862.0  409604.0  111462.0    912.0         0   
      Puerto Rico                0.0    9965.0    1753.0    375.0       242   
      Rhode Island               0.0    9771.0    5823.0    686.0      1571   
 

In [283]:
# Get tables of specific years
df_2011 = df1_by_year.loc['2011']

In [284]:
df_2011

permit   handgun  long_gun    other  multiple  \
month state                                                                    
2011  Alabama                  5036.0  130490.0  146290.0   2121.0      6262   
      Alaska                      0.0   28291.0   34607.0    955.0      1678   
      Arizona                 32906.0  108245.0   81285.0   3111.0      4700   
      Arkansas                30087.0   54659.0   78806.0    768.0      3672   
      California             278625.0  303169.0  320988.0    186.0         0   
      Colorado                 1280.0  148975.0  146166.0      0.0     39875   
      Connecticut             93587.0   49793.0   41435.0   1145.0         0   
      Delaware                    0.0   11126.0   10672.0    266.0       417   
      District of Columbia        0.0     280.0       9.0      0.0         2   
      Florida                    10.0  379889.0  212419.0      0.0     19204   
      Georgia                 90278.0  116794.0  112097.0   1567.0      5137   
      Guam                        0.0     231.0     242.0     21.0        17   
      Hawaii                  12626.0       0.0       0.0      0.0         0   
      Idaho                   16284.0   27967.0   47957.0    687.0      1652   
      Illinois               553984.0  134178.0  133935.0      0.0      6865   
      Indiana                 78242.0  136250.0  130012.0   3860.0      4715   
      Iowa                   135236.0     995.0   41838.0    189.0        53   
      Kansas                  13549.0   60955.0   74165.0   1769.0      3709   
      Kentucky              2019543.0   92936.0  111518.0   1358.0      6362   
      Louisiana                   0.0  101519.0  130125.0   2020.0      4751   
      Maine                       0.0   27040.0   40838.0    663.0      1464   
      Mariana Islands             0.0       0.0       1.0      0.0         0   
      Maryland                  290.0   45851.0   53678.0    318.0       165   
      Massachusetts           83795.0   42506.0   24802.0   1128.0      1237   
      Michigan               201143.0   15224.0  134412.0   1361.0       381   
      Minnesota              124260.0   72073.0  127369.0   2157.0      2776   
      Mississippi              4875.0   56615.0   86909.0   1087.0      3228   
      Missouri                27697.0  167789.0  160145.0   3846.0      7351   
      Montana                  7801.0   29416.0   50377.0    503.0      2508   
      Nebraska                32299.0    1225.0   30806.0    137.0        51   
      Nevada                  14285.0   53246.0   42142.0    113.0      3464   
      New Hampshire           25986.0   33604.0   30808.0    224.0       110   
      New Jersey                  0.0   31163.0   28005.0    231.0       849   
      New Mexico               6413.0   46708.0   45187.0   1240.0      2520   
      New York                37459.0   65774.0  163547.0   3711.0      1032   
      North Carolina         167568.0    3703.0  148662.0   2616.0      2043   
      North Dakota             5801.0   15144.0   36835.0    275.0       721   
      Ohio                        3.0  243338.0  186301.0   5490.0      9949   
      Oklahoma                    1.0  109524.0  109833.0   3327.0      7750   
      Oregon                    578.0   95413.0  106685.0      0.0      7103   
      Pennsylvania             3209.0      94.0  714553.0      3.0      1072   
      Puerto Rico                 0.0    9388.0    1839.0     57.0       241   
      Rhode Island                0.0    8689.0    7241.0    200.0      1141   
      South Carolina          52126.0   81206.0   70232.0   1449.0      2748   
      South Dakota                0.0   20186.0   43602.0    430.0      1291   
      Tennessee               96198.0  188776.0  158193.0      0.0      7940   
      Texas                  161077.0  413943.0  423486.0  10474.0     21885   
      Utah                   939767.0   32105.0   46702.0      6.0      2134   
      Vermont                  